# AutoEncoders

## Importing the libraries

In [1]:
!mkdir ml-1m

In [2]:
!mkdir ml-100k

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [4]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set

In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies

In [6]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

## Converting the data into an array with users in lines and movies in columns

In [7]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network

In [9]:
class SAE(nn.Module):
  def __init__(self,):
    super(SAE,self).__init__()
    self.fc1 = nn.Linear(nb_movies,20) #(number of features,number of nodes in hidden layer)
    self.fc2=nn.Linear(20,10)#fc2 is second full connection
    self.fc3=nn.Linear(10,20)#decoding layer starts
    self.fc4=nn.Linear(20,nb_movies)
    self.activation=nn.Sigmoid()#sigmoid class from nn module
  def forward(self,x):#this function encodes and decodes the data and apply activation functions
    x=self.activation(self.fc1(x))
    x=self.activation(self.fc2(x))
    x=self.activation(self.fc3(x))
    x=self.fc4(x)
    return x
sae=SAE()
criterion= nn.MSELoss() #mean square error loss,you can experiment with optimizers adam or rms
optimizer= optim.RMSprop(sae.parameters(),lr=0.01,weight_decay=0.5)# lr and weight_decay can be changed





## Training the SAE

In [11]:
#optimized code to save memory as much as possible for larger datasets
#advanced technique which is pytorch related
nb_epoch=200
for epoch in range(1,nb_epoch+1):
  train_loss=0
  s=0.
  for id_user in range(nb_users):
    #we add an additional dimension
    input= Variable(training_set[id_user]).unsqueeze(0) #batch of one input vector we are doing online learning,we cannot use simply one dimensional vector pytorch doesn't accept
    target=input.clone()
    if torch.sum(target.data>0)>0: #it means the observation has one non zero rating  #target.data is all the ratings of a user
      output=sae.forward(input)# this is vector of predicted ratings
      target.require_grad=False #we don't use optimization against the target
      output[target ==0] =0 #indices of output vector for which target vector is zero(it saves computation time for large datasets)
      loss=criterion(output,target) #vector of (real ratings,predicted ratings)
      mean_corrector=nb_movies/float(torch.sum(target.data>0)+ 1e-10) #average of non zero ratings
      loss.backward()
      train_loss+=np.sqrt(loss.data * mean_corrector)
      s+=1.
      optimizer.step() #to apply optimizer we use step class
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))#average train_loss among non-zero rating users





epoch: 1 loss: tensor(1.7666)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0311)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0195)
epoch: 11 loss: tensor(1.0188)
epoch: 12 loss: tensor(1.0184)
epoch: 13 loss: tensor(1.0181)
epoch: 14 loss: tensor(1.0173)
epoch: 15 loss: tensor(1.0175)
epoch: 16 loss: tensor(1.0170)
epoch: 17 loss: tensor(1.0168)
epoch: 18 loss: tensor(1.0162)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0160)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0156)
epoch: 25 loss: tensor(1.0157)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0149)
epoch: 29 loss: tensor(1.0128)
epoch: 30 loss: tensor(1.0117)
epoch: 31 loss: tensor(1.0093)
epoch: 32 loss: tensor(1.0094)
epoch: 33 loss: t

## Testing the SAE

In [19]:
#optimized code to save memory as much as possible for larger datasets
#advanced technique which is pytorch related
test_loss=0
s=0.
for id_user in range(nb_users):
#we add an additional dimension
  input= Variable(training_set[id_user]).unsqueeze(0) #batch of one input vector we are doing online learning,we cannot use simply one dimensional vector pytorch doesn't accept
  target=Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data>0)>0: #it means the observation has one non zero rating  #target.data is all the ratings of a user
    output=sae.forward(input)# this is vector of predicted ratings
    target.require_grad=False
    output[target ==0] =0
    loss=criterion(output,target) #vector of (real ratings,predicted ratings)
    mean_corrector=nb_movies/float(torch.sum(target.data>0)+ 1e-10) #average of non zero ratings
    test_loss+=np.sqrt(loss.data * mean_corrector)
    s+=1.
    print(' output'+str(output)+' target'+str(target))
print(' loss: '+str(test_loss/s))#average train_loss among non-zero rating users


 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0.,  ..., 0., 0., 0.]])
 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0.,  ..., 0., 0., 0.]])
 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0.,  ..., 0., 0., 0.]])
 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0.,  ..., 0., 0., 0.]])
 outputtensor([[3.3646, 2.8389, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<IndexPutBackward0>) targettensor([[4., 3., 0.,  ..., 0., 0., 0.]])
 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0.,  ..., 0., 0., 0.]])
 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0.,  ..., 0., 0., 0.]])
 outputtensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<IndexPutBackward0>) targettensor([[0., 0., 0